In [1]:
pip install PyYAML==5.4.1 --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyYAML: filename=PyYAML-5.4.1-cp311-cp311-linux_x86_64.whl size=45657 sha256=9e69247278dfa3608037b73b2727b66bb13957605324dc676437ace2e9d4c6f3
  Stored in directory: /home/jovyan/.cache/pip/wheels/2f/2f/af/a062a8f866dc44812a825da11175d50d7f255900f3b38c79b5
Successfully built PyYAML
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kfp==1.8.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.6/298.6 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 3.0 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. 

In [3]:
!pip install https://storage.googleapis.com/ml-pipeline/release/0.1.29/kfp.tar.gz --upgrade --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 12.8 MB/s eta 0:00:00
  Created wheel for kfp: filename=kfp-0.1.29-py3-none-any.whl size=122684 sha256=9c33be5e6f0ba751c362ea08b79e89f68a8c23a200c9be0e0246ae20c2bbbaa0
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzfqw0sl/wheels/72/45/1d/93aa08a1686673b1250e4cb0a8c86f4344a78fb74c63eb8a0a
  Created wheel for argo-models: filename=argo_models-2.2.1a0-py3-none-any.whl size=57283 sha256=98de42800cae376160d94b5c1bf70b0af9db724420683eee8ba4c24afaca7316

In [4]:
import kfp 
from kfp import dsl

def add_two_numbers(a, b):
    return dsl.ContainerOp(
        name='calculate_sum',
        image='python:3.6.8',
        command=['python', '-c'],
        arguments=['with open("/tmp/results.txt", "a") as file: file.write(str({} + {}))'.format(a, b)],
        file_outputs={
            'data': '/tmp/results.txt',
        }
    )

def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "Result: {}"'.format(text)]
    )

In [5]:
@dsl.pipeline(
  name='Calcualte sum pipeline',
  description='Calculate sum of numbers and prints the result.'
)
def calculate_sum(a=7,b=10,c=4,d=7):
    """A four-step pipeline with first two running in parallel."""

    sum1 = add_two_numbers(a, b)
    sum2 = add_two_numbers(c, d)
    sum = add_two_numbers(sum1.output, sum2.output)

    echo_task = echo_op(sum.output)

In [6]:
kfp.compiler.Compiler().compile(calculate_sum, 'calculate-sum-pipeline.zip')

/opt/conda/lib/python3.11/site-packages/kfp/components/_data_passing.py:227: UserWarning: Missing type name was inferred as "Integer" based on the value "7".
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/kfp/components/_data_passing.py:227: UserWarning: Missing type name was inferred as "Integer" based on the value "10".
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/kfp/components/_data_passing.py:227: UserWarning: Missing type name was inferred as "Integer" based on the value "4".
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/kfp/dsl/_container_op.py:1257: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kf

In [7]:
client = kfp.Client()
aws_experiment = client.create_experiment(name='aws')
my_run = client.run_pipeline(aws_experiment.id, 'calculate-sum-pipeline', 
  'calculate-sum-pipeline.zip')   
